In [203]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn.model_selection import train_test_split
from sklearn import svm
from skimage.feature import hog
import numpy as np
import argparse
import cv2
import os
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn.linear_model import LogisticRegression
from scipy.fft import fft

In [204]:
path_to_a_dataset = r'letters_data_set/A'
path_to_b_dataset = r'letters_data_set/B'
path_to_c_dataset = r'letters_data_set/C'
path_to_d_dataset = r'letters_data_set/D'
path_to_e_dataset = r'letters_data_set/E'
path_to_f_dataset = r'letters_data_set/F'
random_seed=2
NN = KNeighborsClassifier(n_neighbors=15,weights='distance')


In [205]:
def extract_chain_code(img):
    img = cv2.resize(img, (32, 32))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
   
    directions = [0,1,2,3] 
    dx = [0, 1, 0, -1] 
    dy = [-1, 0, 1, 0]
    chain_code = []


# Iterate through each contour
    for cnt in contours:

        # Get coordinates of contour points
        contour = cnt.reshape(-1,2) 
        
        # Start from first point
        x = contour[0][0]
        y = contour[0][1]
        
        # Extract chain code  
        max_iterations = 1000
        while True:

            # Check neighbor points clockwise
            for d in directions:
            
                new_x = x + dx[d] 
                new_y = y + dy[d]  
                
                if new_x in contour[:,0] and new_y in contour[:,1]:
                
                    chain_code.append(d)
                    x, y = new_x, new_y
                    break
            if len(chain_code) > max_iterations:
               break

    
            if x == contour[0][0] and y == contour[0][1]:
                break
    #fft_codes = fft(chain_code) 
    #fft_magnitudes = np.abs(fft_codes) 
    #fft_features = fft_magnitudes / len(chain_code)
    return chain_code



In [206]:
def extract_hog_features(img):
    
    img = cv2.resize(img, (32, 32))
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins,2)
    h = hog.compute(img)
    return h.flatten()

In [207]:
def load_dataset():
    features = []
    labels = []
    img_filenames_a = os.listdir(path_to_a_dataset)
    img_filenames_b = os.listdir(path_to_b_dataset)
    img_filenames_c = os.listdir(path_to_c_dataset)
    img_filenames_d = os.listdir(path_to_d_dataset)
    img_filenames_e = os.listdir(path_to_e_dataset)
    img_filenames_f = os.listdir(path_to_f_dataset)


    for i, fn in enumerate(img_filenames_a):
        if fn.split('.')[-1] != 'png' and fn.split('.')[-1]!='jpg':
            continue

        label = 'A'
        labels.append(label)

        path = os.path.join(path_to_a_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_chain_code(img))
        
        # show an update every 10 images
        if i > 0 and i % 10 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames_a)))
    print("A dataset processing done")
    for i, fn in enumerate(img_filenames_b):
        if fn.split('.')[-1] != 'png' and fn.split('.')[-1]!='jpg':
            continue

        label = 'B'
        labels.append(label)

        path = os.path.join(path_to_b_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_chain_code(img))
        
        # show an update every 10 images
        if i > 0 and i % 10 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames_b)))     
    print("B dataset processing done")   
    
    for i, fn in enumerate(img_filenames_c):
        if fn.split('.')[-1] != 'png' and fn.split('.')[-1]!='jpg':
            continue

        label = 'C'
        labels.append(label)

        path = os.path.join(path_to_c_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_chain_code(img))
        
        # show an update every 10 images
        if i > 0 and i % 10 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames_c)))     
    print("C dataset processing done")   
    
    for i, fn in enumerate(img_filenames_d):
        if fn.split('.')[-1] != 'png':
            continue

        label = 'D'
        labels.append(label)

        path = os.path.join(path_to_d_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_chain_code(img))
        
        # show an update every 10 images
        if i > 0 and i % 10 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames_d)))     
    print("D dataset processing done")   
    
    # for i, fn in enumerate(img_filenames_e):
    #     if fn.split('.')[-1] != 'png':
    #         continue

    #     label = 'E'
    #     labels.append(label)

    #     path = os.path.join(path_to_e_dataset, fn)
    #     img = cv2.imread(path)
    #     features.append(extract_chain_code(img))
        
    #     # show an update every 10 images
    #     if i > 0 and i % 10 == 0:
    #         print("[INFO] processed {}/{}".format(i, len(img_filenames_e)))     
    # print("E dataset processing done")   
    
 
        
    return features, labels        

In [208]:
import numpy as np

def preprocess_features(features, max_length):
    processed_features = []

    for feature in features:
        # Convert scalar to 1D array
        feature = np.atleast_1d(feature)

        # Flatten the feature
        flat_feature = feature.flatten()

        # Pad or truncate to the desired length
        if len(flat_feature) < max_length:
            flat_feature = np.pad(flat_feature, (0, max_length - len(flat_feature)))
        elif len(flat_feature) > max_length:
            flat_feature = flat_feature[:max_length]

        processed_features.append(flat_feature)

    return np.array(processed_features)


In [209]:
def train():
    
    # Load dataset with extracted features
    print('Loading dataset. This will take time ...')
    features, labels = load_dataset()
    print('Finished loading dataset.')
    print(labels)
    features = preprocess_features(features,1000)
    features = features.reshape(features.shape[0], -1)
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.3, random_state=random_seed)
    
    #print(labels)
       
    NN.fit(train_features, train_labels)
        
    
    accuracy = NN.score(test_features, test_labels)
        
    print('accuracy: ', accuracy*100, '%')

In [210]:
train()

Loading dataset. This will take time ...
[INFO] processed 10/45
[INFO] processed 20/45
[INFO] processed 30/45
[INFO] processed 40/45
A dataset processing done
[INFO] processed 10/45
[INFO] processed 20/45
[INFO] processed 30/45
[INFO] processed 40/45
B dataset processing done
[INFO] processed 10/45
[INFO] processed 20/45
[INFO] processed 30/45
[INFO] processed 40/45
C dataset processing done
[INFO] processed 10/45
[INFO] processed 20/45
[INFO] processed 30/45
[INFO] processed 40/45
D dataset processing done
Finished loading dataset.
['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 

In [214]:
letters=['A','B','C','D','E','F']

def natural_sort_key(s):
    """Key function for natural sorting."""
    import re
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]
output_file_path = 'results.txt'
path_to_testset = r'testset3'
filenames = sorted(os.listdir(path_to_testset), key=natural_sort_key)
with open(output_file_path, 'w') as output_file:
    for i, fn in enumerate(filenames):
        if fn.split('.')[-1] != 'png' and fn.split('.')[-1]!='jpg':
            continue
        im=cv2.imread(os.path.join(path_to_testset, fn,))
        #im=cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,75,15)
        cv2.imshow("bounded Literals", im)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        features = extract_chain_code(im)
        features = preprocess_features([features], 1000)
        pred = NN.predict(features)
        result = f"{fn} {letters[np.argmax(pred)]}\n"
        print(pred)
        print(result)  
        output_file.write(result)
# features=extract_hog_features(cv2.imread('test.png'))
# pred=KNN.predict_proba([features])
# print(np.argmax(pred))

['B']
test0.png A

['B']
test1.png A

['D']
test2.png A

['C']
test3.png A

['C']
test4.png A

['D']
test5.png A

['D']
test6.png A

['D']
test7.png A

['D']
test8.png A

['A']
test192.png A

['D']
test193.png A

['D']
test194.png A

['A']
test195.png A

['B']
test196.png A

['B']
test197.png A

